# tsam - 1. Example
Example usage of the time series aggregation module (tsam)
Date: 02.05.2020

Author: Maximilian Hoffmann

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd

import tsam
from tsam import ClusterConfig

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)

Show a slice of the dataset

In [ ]:
raw.head()

Show the shape of the raw input data: 4 types of timeseries (GHI, Temperature, Wind and Load) for every hour in a year

In [ ]:
raw.shape

Create a plot function for the temperature for a visual comparison of the time series

In [ ]:
# Use tsam's built-in plotting with plotly
# tsam.plot_heatmap(data, column, period_hours) creates interactive heatmaps

Plot an example series - in this case the temperature

In [ ]:
# Original temperature heatmap
tsam.plot_heatmap(raw, column="T", period_hours=24, title="Original Temperature")

### Simple k-mean aggregation

Initialize an aggregation class object with k-mean as method for eight typical days, without any integration of extreme periods. Alternative clusterMethod's are 'averaging','hierarchical', 'k_medoids' and 'k_maxoids'.

In [ ]:
result = tsam.aggregate(
    raw,
    n_periods=8,
    period_hours=24,
    cluster=ClusterConfig(method="kmeans"),
)

Create the typical periods

In [ ]:
typPeriods = result.typical_periods

Show shape of typical periods: 4 types of timeseries for 8*24 hours

In [ ]:
typPeriods.shape

Save typical periods to .csv file

In [ ]:
typPeriods.to_csv(os.path.join("results", "testperiods_kmeans.csv"))

Repredict the original time series based on the typical periods

In [ ]:
predictedPeriods = result.reconstruct()

Plot the repredicted data

In [ ]:
# K-means predicted temperature heatmap
tsam.plot_heatmap(
    predictedPeriods, column="T", period_hours=24, title="K-means Predicted Temperature"
)

As seen, they days with the minimal temperature are excluded. In case that they are required they can be added to the aggregation as follow.

### k-maxoids aggregation including extreme periods

Initialize a time series aggregation based on k-maxoids, which automatically searches for points closest to the convex hull.

In [ ]:
resultMax = tsam.aggregate(
    raw,
    n_periods=8,
    period_hours=24,
    cluster=ClusterConfig(method="kmaxoids"),
    rescale=False,
)

Create the typical periods

In [ ]:
typPeriodsMax = resultMax.typical_periods

The aggregation can also be evaluated by indicators

In [ ]:
resultMax.accuracy

Repredict the original time series based on the typical periods

In [ ]:
predictedPeriodsMax = resultMax.reconstruct()

Plot repredicted data

In [ ]:
# K-maxoids predicted temperature heatmap
tsam.plot_heatmap(
    predictedPeriodsMax,
    column="T",
    period_hours=24,
    title="K-maxoids Predicted Temperature",
)

Here bigger biggest values and lower lowest values can be observed compared to k-means clustering.

### Comparison of the aggregations 
It was shown for the temperature, but both times all four time series have been aggregated. Therefore, we compare here also the duration curves  of the electrical load for the original time series, the aggregation with k-mean, and the k-maxoids aggregation.

In [ ]:
# Duration curve comparison using plotly
import plotly.express as px

duration_data = pd.DataFrame(
    {
        "Hour": list(range(len(raw))),
        "Original": raw["Load"].sort_values(ascending=False).values,
        "8 typ days (Centroids)": predictedPeriods["Load"]
        .sort_values(ascending=False)
        .values,
        "8 typ days (Maxoids)": predictedPeriodsMax["Load"]
        .sort_values(ascending=False)
        .values,
    }
)

fig = px.line(
    duration_data.melt(id_vars="Hour", var_name="Method", value_name="Load"),
    x="Hour",
    y="Load",
    color="Method",
    title="Duration Curve Comparison - Load",
)
fig.show()

Or as unsorted time series for an example week

In [ ]:
# Time slice comparison using plotly
time_slice_data = pd.DataFrame(
    {
        "Time": raw["20100210":"20100218"].index,
        "Original": raw["Load"]["20100210":"20100218"].values,
        "8 typ days (Centroids)": predictedPeriods["Load"][
            "20100210":"20100218"
        ].values,
        "8 typ days (Maxoids)": predictedPeriodsMax["Load"][
            "20100210":"20100218"
        ].values,
    }
)

fig = px.line(
    time_slice_data.melt(id_vars="Time", var_name="Method", value_name="Load"),
    x="Time",
    y="Load",
    color="Method",
    title="Time Slice Comparison - Load (Feb 10-18)",
)
fig.show()